Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in _notmist.ipynb_.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [7]:
# Logistic
batch_size = 128

graph=tf.Graph()
with graph.as_default():
    
    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    # Variables.
    weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))
  
    # Training computation.
    logits = tf.matmul(tf_train_dataset, weights) + biases
    beta = 5e-3 #L-2 regularization which penalizes large weights
    R = tf.nn.l2_loss(weights)
    loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) + beta*R
  
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

# let's run it
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
        [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
            valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels)
    

Initialized
Minibatch loss at step 0: 32.474236
Minibatch accuracy: 11.7%
Validation accuracy: 12.1%
Minibatch loss at step 500: 1.756803
Minibatch accuracy: 78.9%
Validation accuracy: 79.7%
Minibatch loss at step 1000: 0.639935
Minibatch accuracy: 85.2%
Validation accuracy: 81.6%
Minibatch loss at step 1500: 0.666970
Minibatch accuracy: 84.4%
Validation accuracy: 81.9%
Minibatch loss at step 2000: 0.868512
Minibatch accuracy: 78.1%
Validation accuracy: 81.3%
Minibatch loss at step 2500: 0.589723
Minibatch accuracy: 86.7%
Validation accuracy: 82.1%
Minibatch loss at step 3000: 0.786721
Minibatch accuracy: 82.8%
Validation accuracy: 81.1%
Test accuracy: 88.4%


In [10]:
# 1-hidden layer neural network, L-2 regularization
batch_size = 128
num_hidden_nodes = 1024

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
 
  # Variables.
  weights_1 = tf.Variable(tf.truncated_normal([image_size * image_size, num_hidden_nodes]))
  biases_1 = tf.Variable(tf.zeros([num_hidden_nodes]))
  weights_2 = tf.Variable(tf.truncated_normal([num_hidden_nodes, num_labels]))
  biases_2 = tf.Variable(tf.zeros([num_labels]))
    
  # Training computation.  
  x1 = tf.matmul(tf_train_dataset, weights_1) + biases_1
  hidden_layer = tf.nn.relu(x1)
  logits = tf.matmul(hidden_layer, weights_2) + biases_2
  beta = 1e-3  #L-2 regularization which penalizes large weights
  R = tf.nn.l2_loss(weights_1) + tf.nn.l2_loss(weights_2)
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))+beta*R
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  
  valid_hidden_layer = tf.nn.relu(tf.matmul(tf_valid_dataset, weights_1) + biases_1)  
  valid_prediction = tf.nn.softmax(tf.matmul(valid_hidden_layer, weights_2) + biases_2)
   
  test_hidden_layer = tf.nn.relu(tf.matmul(tf_test_dataset, weights_1) + biases_1)
  test_prediction = tf.nn.softmax(tf.matmul(test_hidden_layer, weights_2) + biases_2)

    
# run model
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
            [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
            valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 636.325439
Minibatch accuracy: 8.6%
Validation accuracy: 22.5%
Minibatch loss at step 500: 192.115173
Minibatch accuracy: 79.7%
Validation accuracy: 81.0%
Minibatch loss at step 1000: 116.989456
Minibatch accuracy: 87.5%
Validation accuracy: 80.1%
Minibatch loss at step 1500: 68.416336
Minibatch accuracy: 85.2%
Validation accuracy: 82.5%
Minibatch loss at step 2000: 41.717876
Minibatch accuracy: 80.5%
Validation accuracy: 84.7%
Minibatch loss at step 2500: 25.048573
Minibatch accuracy: 93.8%
Validation accuracy: 85.8%
Minibatch loss at step 3000: 15.476253
Minibatch accuracy: 87.5%
Validation accuracy: 87.0%
Test accuracy: 93.1%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

To create overfitting, replace step by 1 so that we will run the same batch multiple times. The validation accuracy does not increase as the training time increases.

In [11]:
# 1-hidden layer neural network, run the same batch to create overfitting
batch_size = 128
num_hidden_nodes = 1024

graph=tf.Graph()
with graph.as_default():
    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
 
    # Variables.
    weights_1 = tf.Variable(tf.truncated_normal([image_size * image_size, num_hidden_nodes]))
    biases_1 = tf.Variable(tf.zeros([num_hidden_nodes]))
    weights_2 = tf.Variable(tf.truncated_normal([num_hidden_nodes, num_labels]))
    biases_2 = tf.Variable(tf.zeros([num_labels]))
    
    # Training computation.  
    x1 = tf.matmul(tf_train_dataset, weights_1) + biases_1
    hidden_layer = tf.nn.relu(x1)
    logits = tf.matmul(hidden_layer, weights_2) + biases_2
    beta = 0.001  #L-2 regularization which penalizes large weights
    R = tf.nn.l2_loss(weights_1) + tf.nn.l2_loss(weights_2)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))+beta*R
  
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
  
    valid_hidden_layer = tf.nn.relu(tf.matmul(tf_valid_dataset, weights_1) + biases_1)  
    valid_prediction = tf.nn.softmax(tf.matmul(valid_hidden_layer, weights_2) + biases_2)
   
    test_hidden_layer = tf.nn.relu(tf.matmul(tf_test_dataset, weights_1) + biases_1)
    test_prediction = tf.nn.softmax(tf.matmul(test_hidden_layer, weights_2) + biases_2)

# run model
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
        offset = (1 * batch_size) % (train_labels.shape[0] - batch_size) # create overfitting
    # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
        [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
        print("Minibatch loss at step %d: %f" % (step, l))
        print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
        print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 3000: 15.657056
Minibatch accuracy: 100.0%
Validation accuracy: 65.8%
Test accuracy: 72.6%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [12]:
# 1-hidden layer neural network, run the same batch to create overfitting
batch_size = 128 
num_hidden_nodes = 1024

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
 
  # Variables.
  weights_1 = tf.Variable(tf.truncated_normal([image_size * image_size, num_hidden_nodes]))
  biases_1 = tf.Variable(tf.zeros([num_hidden_nodes]))
  weights_2 = tf.Variable(tf.truncated_normal([num_hidden_nodes, num_labels]))
  biases_2 = tf.Variable(tf.zeros([num_labels]))
    
  # Training computation.  
  x1 = tf.matmul(tf_train_dataset, weights_1) + biases_1
  hidden_layer = tf.nn.relu(tf.nn.dropout(x1, 0.5)) # drop out
  logits = tf.matmul(hidden_layer, weights_2) + biases_2
  beta = 0.001  #L-2 regularization which penalizes large weights
  R = tf.nn.l2_loss(weights_1) + tf.nn.l2_loss(weights_2)
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))+beta*R
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  
  valid_hidden_layer = tf.nn.relu(tf.matmul(tf_valid_dataset, weights_1) + biases_1)  
  valid_prediction = tf.nn.softmax(tf.matmul(valid_hidden_layer, weights_2) + biases_2)
   
  test_hidden_layer = tf.nn.relu(tf.matmul(tf_test_dataset, weights_1) + biases_1)
  test_prediction = tf.nn.softmax(tf.matmul(test_hidden_layer, weights_2) + biases_2)

    
# run model
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (1 * batch_size) % (train_labels.shape[0] - batch_size) # overfitting
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))



Initialized
Minibatch loss at step 0: 737.930969
Minibatch accuracy: 13.3%
Validation accuracy: 27.2%
Minibatch loss at step 500: 191.518768
Minibatch accuracy: 100.0%
Validation accuracy: 67.9%
Minibatch loss at step 1000: 116.155602
Minibatch accuracy: 100.0%
Validation accuracy: 67.6%
Minibatch loss at step 1500: 70.449234
Minibatch accuracy: 100.0%
Validation accuracy: 67.4%
Minibatch loss at step 2000: 42.726570
Minibatch accuracy: 100.0%
Validation accuracy: 67.8%
Minibatch loss at step 2500: 25.912374
Minibatch accuracy: 100.0%
Validation accuracy: 66.7%
Minibatch loss at step 3000: 15.715013
Minibatch accuracy: 100.0%
Validation accuracy: 68.3%
Test accuracy: 74.8%


Using dropout increases the test accuracy from 72.6% to 74.8% for the overfitting case.

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [13]:
# 1-hidden layer neural network, L-2 regularization, learning rate decay
batch_size = 128
num_hidden_nodes = 1024

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
 
  # Variables.
  weights_1 = tf.Variable(tf.truncated_normal([image_size * image_size, num_hidden_nodes]))
  biases_1 = tf.Variable(tf.zeros([num_hidden_nodes]))
  weights_2 = tf.Variable(tf.truncated_normal([num_hidden_nodes, num_labels]))
  biases_2 = tf.Variable(tf.zeros([num_labels]))
    
  # Training computation.  
  x1 = tf.matmul(tf_train_dataset, weights_1) + biases_1
  hidden_layer = tf.nn.relu(x1)
  logits = tf.matmul(hidden_layer, weights_2) + biases_2
  beta = 1e-3  #L-2 regularization which penalizes large weights
  R = tf.nn.l2_loss(weights_1) + tf.nn.l2_loss(weights_2)
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))+beta*R
  
  # Optimizer, learning rate decay
  global_step = tf.Variable(0)  # count the number of steps taken.
  learning_rate = tf.train.exponential_decay(0.3, global_step, 100000, 0.96)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
    
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  
  valid_hidden_layer = tf.nn.relu(tf.matmul(tf_valid_dataset, weights_1) + biases_1)  
  valid_prediction = tf.nn.softmax(tf.matmul(valid_hidden_layer, weights_2) + biases_2)
   
  test_hidden_layer = tf.nn.relu(tf.matmul(tf_test_dataset, weights_1) + biases_1)
  test_prediction = tf.nn.softmax(tf.matmul(test_hidden_layer, weights_2) + biases_2)

    
# run model
num_steps = 30001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 3000 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 721.941162
Minibatch accuracy: 2.3%
Validation accuracy: 23.0%
Minibatch loss at step 3000: 50.915806
Minibatch accuracy: 81.2%
Validation accuracy: 85.0%
Minibatch loss at step 6000: 8.800516
Minibatch accuracy: 88.3%
Validation accuracy: 88.3%
Minibatch loss at step 9000: 1.883548
Minibatch accuracy: 89.1%
Validation accuracy: 89.0%
Minibatch loss at step 12000: 0.759922
Minibatch accuracy: 89.1%
Validation accuracy: 89.5%
Minibatch loss at step 15000: 0.435598
Minibatch accuracy: 93.8%
Validation accuracy: 89.6%
Minibatch loss at step 18000: 0.420610
Minibatch accuracy: 93.0%
Validation accuracy: 89.2%
Minibatch loss at step 21000: 0.346873
Minibatch accuracy: 93.8%
Validation accuracy: 89.5%
Minibatch loss at step 24000: 0.536510
Minibatch accuracy: 87.5%
Validation accuracy: 89.5%
Minibatch loss at step 27000: 0.378691
Minibatch accuracy: 93.0%
Validation accuracy: 89.5%
Minibatch loss at step 30000: 0.591975
Minibatch accuracy: 85.2%
Validati